# Basic Elo Predictor

Calculates ELO scores based on all historical matches, allows for customizing k factor, inital elo, and reversion ratio but not much else.

Then predicts the outcome using the win % formula. 

In [39]:
import pandas as pd
import yaml
import duckdb as db
import warnings

with open('config.yaml', 'r') as file:
    config_file = yaml.safe_load(file)
data_dir = config_file.get("data_dir")
output_dir = config_file.get("output_dir")


In [7]:
submission_df = pd.read_csv(f'{data_dir}/Kaggle/SampleSubmissionStage2.csv')

def extract_game_info(id_str):
    # Extract year and team_ids
    parts = id_str.split('_')
    year = int(parts[0])
    teamID1 = int(parts[1])
    teamID2 = int(parts[2])
    return year, teamID1, teamID2

submission_df[['Season', 'TeamID1', 'TeamID2']] = submission_df['ID'].apply(extract_game_info).tolist()

In [ ]:
# Some men's teams leave D1, so we *should* filter them out, but it breaks the basic model so I ignore it. The mean is still 1500.
# mensids = db.sql('FROM "./SourceData/Kaggle/MTeams.csv" WHERE LastD1Season = 2025').to_df()
mensids = db.sql('FROM "./SourceData/Kaggle/MTeams.csv"').to_df()

womensids = db.sql('FROM "./SourceData/Kaggle/WTeams.csv" ').to_df()

In [9]:
womens_results = pd.read_csv(f'{data_dir}/Kaggle/WRegularSeasonCompactResults.csv')
mens_results = pd.read_csv(f'{data_dir}/Kaggle/MRegularSeasonCompactResults.csv')

In [14]:
k_set = 30 # K-factor for Elo rating
# K-factor determines how much the Elo rating changes after each game
# Higher K-factor means more volatility in Elo ratings, Lower K-factor means more stable Elo ratings
# The K-factor is usually set between 10 and 40, 30 being the standard
initial_elo_set = 1500 # Initial Elo rating for all teams
# Initial Elo rating is usually set to 1500, but can be set to any value
mean_reversion = .25 # Mean reversion ratio for Elo rating
# Mean reversion ratio determines how much the Elo rating reverts to the initial/mean Elo rating after each season
# Used to reflect the turnover in a sports team, higher mean reversion means more turnover in the team
# I chose to go with 25% of the elo returns to mean as a starting point

def update_elo(winner_elo, loser_elo, k=k_set):
    expected_win = 1 / (1 + 10**((loser_elo - winner_elo) / 400))
    new_winner_elo = winner_elo + k * (1 - expected_win)
    new_loser_elo = loser_elo - k * (1 - expected_win)
    return new_winner_elo, new_loser_elo

def run_basic_elo(season_results_df, ids_df):

    seasons_array = sorted(season_results_df['Season'].unique())
    initial_elo = initial_elo_set 
    elo_ratings = {team_id: initial_elo for team_id in ids_df['TeamID'].unique()}

    for i in seasons_array:
        results_season = season_results_df[season_results_df['Season'] == i]
        # print(i)
        for index, row in results_season.iterrows():
            winner = row['WTeamID']
            loser = row['LTeamID']
            if row['WLoc'] == 'H':
                winner_elo = elo_ratings[winner] + 100
            elif row['WLoc'] == 'A':
                loser_elo = elo_ratings[loser] + 100
                
            winner_elo = elo_ratings[winner]
            loser_elo = elo_ratings[loser]
            new_winner_elo, new_loser_elo = update_elo(winner_elo, loser_elo)
            elo_ratings[winner] = new_winner_elo
            elo_ratings[loser] = new_loser_elo
        elo_ratings = {team_id: (1-mean_reversion) * elo + (mean_reversion) * initial_elo for team_id, elo in elo_ratings.items()}
        df = pd.DataFrame(list(elo_ratings.items()), columns=['TeamID', 'Elo'])

    return df

mens_elo = run_basic_elo(mens_results, mensids)
womens_elo = run_basic_elo(womens_results, womensids)

In [ ]:
# Double check the mean elo is still ~1500 without handling the teams that leave D1
mensids.merge(mens_elo, on='TeamID', how='left').query("LastD1Season == 2025")['Elo'].mean()

np.float64(1500.6108274685903)

In [30]:
All_elo = pd.concat([mens_elo, womens_elo], ignore_index=True)

warnings.filterwarnings('ignore')
# Create a dictionary for quick lookup of ELO ratings by TeamID
elo_dict = All_elo.set_index('TeamID')['Elo'].to_dict()

# Map the ELO ratings to the TeamID1 column in the submission_df
submission_df['TeamID1_Elo'] = submission_df['TeamID1'].map(elo_dict)
submission_df['TeamID2_Elo'] = submission_df['TeamID2'].map(elo_dict)

# Fill missing values with 9999 - these would be teams that aren't in the nate database of mismatches in names
submission_df['TeamID1_Elo'].fillna(9999, inplace=True)
submission_df['TeamID2_Elo'].fillna(9999, inplace=True)

# Check the result, this should be 0
assert len(submission_df.query('TeamID1_Elo == 9999 or TeamID2_Elo == 9999')) == 0, "There are teams with missing ELO ratings"

In [ ]:
# Basic ELO win probability calculation
def calc_elo_win(A, B):
    awin = 1 / (1 + 10**( (B - A) / 400))
    return(awin)
submission_df['Team1_win_prob'] = submission_df.apply(lambda x: calc_elo_win(x['TeamID1_Elo'], x['TeamID2_Elo']), axis=1)

In [33]:
submission_df

,ID,Pred,Season,TeamID1,TeamID2,TeamID1_Elo,TeamID2_Elo,Team1_win_prob
0,2025_1101_1102,0.5,2025,1101,1102,1474.486568,1341.858894,0.682106
1,2025_1101_1103,0.5,2025,1101,1103,1474.486568,1633.269184,0.286177
2,2025_1101_1104,0.5,2025,1101,1104,1474.486568,1745.290165,0.173809
3,2025_1101_1105,0.5,2025,1101,1105,1474.486568,1320.744042,0.707860
4,2025_1101_1106,0.5,2025,1101,1106,1474.486568,1393.934144,0.613891
...,...,...,...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479,1337.393918,1351.195809,0.480148
131403,2025_3477_3480,0.5,2025,3477,3480,1337.393918,1446.435133,0.348034
131404,2025_3478_3479,0.5,2025,3478,3479,1350.706730,1351.195809,0.499296
131405,2025_3478_3480,0.5,2025,3478,3480,1350.706730,1446.435133,0.365619


In [40]:
Output = submission_df[['ID', 'Team1_win_prob']].rename(columns={'Team1_win_prob': 'Pred'})
Output.to_csv(f'{output_dir}/BasicEloProbs.csv', index=False)